In [236]:
require 'torch';
require 'nn';
require 'optim';

### Prepair data

In [237]:
local matio = require 'matio'
data = matio.load('ex4data1.mat')

In [238]:
trainset = {}
trainset.data = data.X
trainset.label = data.y[{ {}, 1}]

In [239]:
trainset

In [240]:
setmetatable(trainset,
    {__index = function(t,i)
                return {t.data[i], t.label[i]}
        end}
);
 
function trainset:size()
    return self.data:size(1)
end

{
  data : DoubleTensor - size: 5000x400
  label : DoubleTensor - size: 5000
}


In [241]:
mean = {}
stdv = {}
for i=1,400 do
    mean[i] = trainset.data[{ {},{i} }]:mean()
    stdv[i] = trainset.data[{ {}, {i} }]:std()
    --print(i .. 'th mean: ' .. mean[i])
    --print(i .. 'th std dev: ' .. stdv[i])
    trainset.data[{ {},{i} }]:add(-mean[i])
    if stdv[i] ~= 0 then
        trainset.data[{ {},{i} }]:div(stdv[i])
    end
end

### define model

In [242]:
n_train_data = trainset:size(1) -- number of training data
n_inputs = trainset:size(2)     -- number of cols = number of dims of input
n_outputs = 10   -- highest label = # of classes

In [243]:
net = nn.Sequential()
net:add(nn.Linear(400,25))
net:add(nn.Sigmoid())
net:add(nn.Linear(25,10))
net:add(nn.Sigmoid())
net:add(nn.LogSoftMax())

### define loss function

In [244]:
opt = {
    optimization = 'sgd',
    batch_size = 5000,
    train_size = 5000,  -- set to 0 or 60000 to use all 60000 training data
    test_size = 0,      -- 0 means load all data
    epochs = 1e3,         -- **approximate** number of passes through the training data (see below for the `iterations` variable, which is calculated from this)
}         -- these options are used throughout


In [245]:
criterion = nn.ClassNLLCriterion()

In [246]:
parameters, gradParameters = net:getParameters()

In [247]:
counter = 0
feval = function(x)
  if x ~= parameters then
    parameters:copy(x)
  end

  gradParameters:zero()
    
  local batch_inputs = trainset.data[{{}, {}}]
  local batch_targets = trainset.label[{{}}]

  batch_outputs = net:forward(batch_inputs)
  batch_loss = criterion:forward(batch_outputs, batch_targets)
  dloss_doutput = criterion:backward(batch_outputs, batch_targets) 
  net:backward(batch_inputs, dloss_doutput)

  return batch_loss, gradParameters
end  

### train

In [248]:
optimState = {
    learningRate = 5,
    weightDecay = 0,
    momentum = 0,
    learningRateDecay = 1e-2
}
optimMethod = optim.sgd

In [249]:
losses = {}          -- training losses for each iteration/minibatch
epochs = opt.epochs  -- number of full passes over all the training data
iterations = epochs * math.ceil(n_train_data / opt.batch_size) -- integer number of minibatches to process

for i = 1, iterations do
  local _, minibatch_loss = optimMethod(feval, parameters, optimState)

  if i % 10 == 1 then -- don't print *every* iteration, this is enough to get the gist
      print(string.format("minibatches processed: %6s, loss = %6.6f", i, minibatch_loss[1]))
  end
  losses[#losses + 1] = minibatch_loss[1] -- append the new loss
end


minibatches processed:      1, loss = 2.302787	


minibatches processed:     11, loss = 2.008609	


minibatches processed:     21, loss = 1.847087	


minibatches processed:     31, loss = 1.771381	


minibatches processed:     41, loss = 1.727683	


...

minibatches processed:    951, loss = 1.539984	


minibatches processed:    961, loss = 1.539781	


minibatches processed:    971, loss = 1.539581	


minibatches processed:    981, loss = 1.539384	


minibatches processed:    991, loss = 1.539190	


### test

In [250]:
correction = 0
for i=1,trainset:size() do
    local answer = trainset.label[i]
    local prediction = net:forward(trainset.data[i])
    local confidences, indices = torch.sort(prediction, true)
    if (answer == indices[1]) then
        correction = correction + 1
    end
end
print(correction, 100*correction/trainset:size() .. '%')

4746	94.92%	
